In [ ]:
###### Load data

### Data preproccesing

In [1]:
###### Build pretrained data

In [2]:
# combinate month, shop and item in order of increasing month
pretrained_data  = []
cols  = ["date_block_num", "shop_id", "item_id"]
for i in range(34):
    sales = data['sales_train'][data['sales_train'].date_block_num == i]
    pretrained_data.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique()))))

pretrained_data = mpd.DataFrame(np.vstack(pretrained_data), columns = cols )
pretrained_data.sort_values(cols, inplace = True)

# merge datasets for data preprocessing
pretrained_data = mpd.merge(pretrained_data, data['shops'], on = ["shop_id"], how = "left" )
pretrained_data = mpd.merge(pretrained_data, data['items'], on = ["item_id"], how = "left")
pretrained_data = mpd.merge(pretrained_data, data['item_categories'], on = ["item_category_id"], how = "left" )

NameError: name 'data' is not defined

In [3]:
# add profit from sales
pretrained_data["profit"] = data['sales_train']["item_cnt_day"] * data['sales_train']["item_price"]

NameError: name 'data' is not defined

In [ ]:
# add count of sold items per month
group = data['sales_train'].groupby(["date_block_num", "shop_id", "item_id"]).agg({"item_cnt_day": ["sum"]})
group.columns = ["item_cnt_month"]
group.reset_index(inplace = True)
pretrained_data = mpd.merge(pretrained_data, group, on = cols, how = "left")
pretrained_data["item_cnt_month"] = pretrained_data["item_cnt_month"].fillna(0)

In [ ]:
pretrained_data.head(2)

In [ ]:
###### Substruct some categories from string data 

In [ ]:
# item categories data
pretrained_data["type_code"] = pretrained_data.item_category_name.apply(lambda x: x.split(" ")[0]).astype(str)
pretrained_data.type_code = LabelEncoder().fit_transform(pretrained_data.type_code)

In [ ]:
pretrained_data.head(2)

In [ ]:
pretrained_data["split"] = pretrained_data.item_category_name.apply(lambda x: x.split("-"))
pretrained_data["subtype"] = pretrained_data.split.apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
pretrained_data["subtype_code"] = LabelEncoder().fit_transform(pretrained_data["subtype"])
pretrained_data = pretrained_data.drop(columns=['item_category_name', 'split', 'subtype'])

In [ ]:
pretrained_data.head(2)

In [ ]:
%%time
# shops data
pretrained_data["shop_city"] = pretrained_data.shop_name.str.split(" ").map(lambda x: x[0])
pretrained_data["shop_category"] = pretrained_data.shop_name.str.split(" ").map(lambda x: x[1])
pretrained_data["shop_category"] = LabelEncoder().fit_transform(pretrained_data.shop_category)
pretrained_data["shop_city"] = LabelEncoder().fit_transform(pretrained_data.shop_city)
pretrained_data = pretrained_data.drop(columns=['shop_name'])

In [ ]:
pretrained_data.head(2)

In [ ]:
# item data
pretrained_data["name1"], pretrained_data["name2"] = pretrained_data.item_name.str.split('[', n=1).str[0], pretrained_data.item_name.str.split('[', n=1).str[1]
pretrained_data["name1"], pretrained_data["name3"] = pretrained_data.item_name.str.split('(', n=1).str[0], pretrained_data.item_name.str.split('(', n=1).str[1]

# replace special characters and turn to lower case
pretrained_data["name2"] = pretrained_data.name2.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
pretrained_data["name3"] = pretrained_data.name3.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

pretrained_data = pretrained_data.fillna('0')

pretrained_data.name2 = LabelEncoder().fit_transform(pretrained_data.name2)
pretrained_data.name3 = LabelEncoder().fit_transform(pretrained_data.name3)

pretrained_data = pretrained_data.drop(columns=['name1', 'item_name']) 

In [ ]:
pretrained_data.head(2)

In [ ]:
###### time series features

In [ ]:
def extract_lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = mpd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
%%time
# item count per month lag
pretrained_data = extract_lag_feature(pretrained_data, [1, 2, 3, 12], 'item_cnt_month')

In [ ]:
pretrained_data.head()

In [ ]:
# lag of average item count sales per month 

group = pretrained_data.groupby(["date_block_num"] ).agg({"item_cnt_month" : "mean"})
group.columns = ["avg_by_month_item_cnt"]
group.reset_index()


pretrained_data = mpd.merge(pretrained_data, group, on = ["date_block_num"], how = "left")
del(group)
pretrained_data = extract_lag_feature(pretrained_data, [1, 2, 3], "avg_by_month_item_cnt")
pretrained_data.drop(["avg_by_month_item_cnt"], axis = 1, inplace = True)

In [ ]:
pretrained_data.head(2)

In [ ]:
# lag of average item sales per month of each id

group = pretrained_data.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['avg_by_month_item_id_item_cnt']
group = group.reset_index()


pretrained_data = mpd.merge(pretrained_data, group, on=['date_block_num', 'item_id'], how='left')
del(group)
pretrained_data = extract_lag_feature(pretrained_data, [1, 2, 3, 12], "avg_by_month_item_id_item_cnt")
pretrained_data.drop(columns = ['avg_by_month_item_id_item_cnt'], axis = 1, inplace = True)

In [ ]:
pretrained_data.tail(2)

In [ ]:
# lag of average item sales per month of each shop

group = pretrained_data.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['avg_by_month_shop_item_cnt']
group = group.reset_index()

pretrained_data = mpd.merge(pretrained_data, group, on=['date_block_num', 'shop_id'], how='left')

pretrained_data = extract_lag_feature(pretrained_data, [1,2,3], "avg_by_month_shop_item_cnt")
pretrained_data.drop(columns = ['avg_by_month_shop_item_cnt'], axis = 1, inplace = True)

In [ ]:
pretrained_data.tail(2)

In [ ]:
# lag of average item sales per month of each city

group = pretrained_data.groupby(['date_block_num', 'shop_city']).agg({'item_cnt_month': 'mean'})
group.columns = ['avg_by_month_city_item_cnt']
group = group.reset_index()


pretrained_data = mpd.merge(pretrained_data, group, on=['date_block_num', 'shop_city'], how='left')
del(group)
pretrained_data = extract_lag_feature( pretrained_data, [1], "avg_by_month_city_item_cnt" )
pretrained_data.drop(columns = ['avg_by_month_city_item_cnt'], axis = 1, inplace = True)

In [ ]:
pretrained_data.tail(2)

In [ ]:
# lag of average item sales per month of each category

group = pretrained_data.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['avg_by_month_cat_item_cnt']
group = group.reset_index()


pretrained_data = mpd.merge(pretrained_data, group, on=['date_block_num', 'item_category_id'], how='left')
del(group)
pretrained_data = extract_lag_feature(pretrained_data, [1], "avg_by_month_cat_item_cnt")
pretrained_data.drop(columns = ['avg_by_month_cat_item_cnt'], axis = 1, inplace = True)

In [ ]:
pretrained_data.tail(2)

In [ ]:
###### Add clusters features

In [ ]:
# price clusters

In [ ]:
# shop clusters

In [ ]:
# categories cluster

In [ ]:
# split good_pairs only item_id info?

In [ ]:
###### Other usefull features

1. mean of price by month
2. item first sale
3. item last sale
4. change of sales of item/price by month